## 第3章 確率的勾配降下法

### 3.1 勾配降下法

勾配：

$$
\nabla E \equiv \frac{\partial E}{\partial {\bf w}} = \left[ \frac{\partial E}{\partial w_1} \frac{\partial E}{\partial w_2} \dots \frac{\partial E}{\partial w_M} \right]^T
$$

勾配降下法：

${\bf w}$ を「負の勾配方向 $-\nabla E$ に少し動かす」ことを繰り返して、その極小点を求める方法。

$$ {\bf w}^{(t+1)} = {\bf w}^{(t)} - \epsilon \nabla E $$

この $\epsilon$ を **学習係数**（**learning rate**）と呼ぶ。

+ $\epsilon$ が小さい ⇒ 学習にかかる時間が大きくなる
+ $\epsilon$ が大きい ⇒ 極小点に収束しないことがある

※ TensorFlow では `tf.train.GradientDescentOptimizer` というクラスが用意されている。

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.01) # 引数は学習係数（learning rate）
train_step = optimizer.minimize(cross_entropy)

### 3.2 確率的勾配降下法

**バッチ学習**（**batch learning**）（もしくは **エポック学習**（**epoch learning**））：  
3.1 節で見た（全訓練データを利用した）勾配降下法による学習。

**確率的勾配降下法**（**stochastic gradient descent**）（もしくは **逐次的勾配降下法**（**sequential gradient descent**））：  
訓練データいくつか（極端には1つ）ずつ繰り返し適用していく勾配降下法。  
**SGD** と略される。

$$
E({\bf w}) = \sum_n E_n({\bf w}) \\
{\bf w}^{(t+1)} = {\bf w}^{(t)} - \epsilon \nabla E_n 
$$

| | バッチ学習 | SGD |
| --- | --- | --- |
| **冗長性への対応** | データ量に比例して計算コストがかかる | 学習内容・計算量に影響なし |
| **局所解** | 望まない局所的な極小解に陥るリスクが高い | 局所的な極小解に陥るリスクが低い |

その他 SGD の利点：

+ 学習の途中経過を随時確認出来る。
+ オンライン学習（＝データの収集と最適化の計算を同時並行に行える）

※ TensorFlow では、`tf.train.GradientDescentOptimizer` から `train_step` を定義したら、それをループで訓練データ1つずつ適用していけば SGD が実現出来る。

### 3.3 ミニバッチ

**ミニバッチ**（**minibatch**）:  
SGD を「訓練データ1つずつ」ではなく「いくつか（10〜100）ずつ」適用する場合の、そのひとまとめにしたサンプル集合。

※ TensorFlow では、`tf.train.GradientDescentOptimizer` から `train_step` を定義したら、バッチサイズを決めてループで訓練データをその個数ずつ取り出して適用していけば ミニバッチによるSGD が実現出来る。  

コード例：

In [ ]:
for i in range(1000):
    batch_xs, batch_ys = training_data.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

### 3.4 汎化性能と過適合

**訓練誤差**（**training error**）：  
訓練データに対する誤差（＝学習結果（もしくは途中経過）の誤差関数の値）

**汎化誤差**（**generalization error**）：  
サンプルの母集団に対する誤差（≒未知のデータに対してどれくらい正しい推定が行えているか）の期待値

**テスト誤差**（**test error**）：  
訓練データとは別に用意した**テストデータ**に対する誤差（＝学習結果のモデルによる誤差関数の値）  
汎化誤差の目安として利用

**過適合**（**overfitting**）（もしくは**過学習**（**overlearning**））：  
訓練データに過剰に適合するように学習されて、未知のデータ（もしくはテストデータ）にうまく適合しなくなっている状態。  
**学習曲線**を描いたとき、テスト誤差が訓練誤差と乖離してしまっている状態。  
テスト誤差が乖離し始めたら学習を打ち切ることを**早期終了**（**early stopping**）（あるいは**早期打ち切り**）と呼ぶ。

### 3.5 過適合の緩和

#### 3.5.1 正則化

**正則化**（**regularization**）：  
過適合を緩和するための（いくつかの）手法

+ 重み減衰
+ 重み上限
+ ドロップアウト

#### 3.5.2 重みの制約

**重み減衰**（**weight decay**）：  
誤差関数に重みの二乗和（二乗ノルム）を加算する正則化手法。

$$
E_t({\bf w}) \equiv \frac{1}{N_t} \sum_{n \in D_t} E_n({\bf w}) + \frac{\lambda}{2}\|{\bf w}\|^2
\\
{\bf w}^{(t+1)} = {\bf w}^{(t)} - \epsilon \left(\frac{1}{N_t} \nabla E_n + \lambda {\bf w}^{(t)} \right)
$$

（$\lambda$：0.01〜0.00001 くらいの範囲の定数（正則化パラメータと呼ぶこともある））

**重み上限**：  
重みの大きさの上限を制約する正則化手法（詳細略）。  
※重み減衰よりを上回る効果がある（らしい）。

#### 3.5.3 ドロップアウト

**ドロップアウト**（**dropout**）：  
多層ネットワークのユニットを確率的に選別して学習する方法（詳細略）。  

### 3.6 学習のトリック

#### 3.6.1 データの正規化

**正規化**（**normalization**）（もしくは**標準化**（**standardization**））：  
データの平均を0に、分散を1になるよう変換すること。

#### 3.6.2 データ拡張

#### 3.6.3 複数ネットの平均

#### 3.6.4 学習係数の決め方

#### 3.6.5 モメンタム

#### 3.6.6 重みの初期化

#### 3.6.7 サンプルの順序